### Scraper
- This part of the code handles getting the individual player information which is hosted at [Stats Royale](!https://statsroyale.com/)

### Before Scraping
**IMPORTANT** the information pulled is only as accurate as the information reflected on the website. To ensure the player information is accurate go to the player id and refresh the page
- [Dad](!https://statsroyale.com/profile/L8RCCJGV)
- [Carter](!https://statsroyale.com/profile/PVQ90YCV)
- [Chris](!https://statsroyale.com/profile/2JV9RJYJG)

In [1]:
from bs4 import BeautifulSoup
import requests
from pandas import ExcelWriter, DataFrame
import pandas as pd
import datetime

In [2]:
CARTER_KEY = "PVQ90YCV"
CHRIS_KEY = "2JV9RJYJG"
DAD_KEY = "L8RCCJGV"

rarities = ['Common', 'Epic', 'Legendary', 'Rare']

In [3]:
# Return parsed profile page using BS4
def parseURL(tag, sort_by):
    if sort_by == "level":
         link = "https://statsroyale.com/profile/{}/cards?sort=level".format(tag)
    elif sort_by == "elixir":
        link = "https://statsroyale.com/profile/{}/cards?sort=exlixir".format(tag)
    elif sort_by == "rarity":
        link = "https://statsroyale.com/profile/{}/cards?sort=rarity".format(tag)
    elif sort_by == "arena":
        link = "https://statsroyale.com/profile/{}/cards?sort=arena".format(tag)
    
    response = requests.get(link).text
    soup = BeautifulSoup(response, 'html.parser')
    return soup

In [4]:
class Card:
    ''' 
    Class used to hold all the information relevant to each individual card
    '''
    def __init__(self, card_html):
        self.name, self.level, self.curr_count, self.rarity = self.parse(card_html)
    
    def parse(self, card_html):
        rarity_map = {
            "1" : "Common", 
            "2" : "Rare",
            "3" : "Epic",
            "4" : "Legendary"
        }
        name = card_html.find("div", {"class" : "ui__tooltip ui__tooltipTop ui__tooltipMiddle cards__tooltip"}).text.replace('\n','')
        level = card_html.find("a").text.replace('\n','')
        
        if level == "Max Lvl":
            level = "13"
        else:
            level = level[4:]
            if level == "":
                level = "0"
        try:
            curr_count = card_html.find("div", {"class" : "profileCards__meter__numbers"}).text.replace('\n','')
        except:
            curr_count = 0 # this occurs when the user does not have the card yet
        rarity = card_html["data-rarity"][0]
        return name, level, curr_count, rarity_map[rarity]
    
    
    def __repr__(self):
        return "{}".format(self.name)
    
    def __hash__(self):
        return hash(self.name)

    def to_row(self):
        return {"Name" : self.name, "Level": self.level, "Count" : self.curr_count, "Rarity" : self.rarity}

In [5]:
def to_df(key, verbose=False):
    carter_soup = parseURL(key, "rarity")
    carter_cards_html = carter_soup.findAll("div", {"class": "profileCards__card upgrade "}) + \
                        carter_soup.findAll("div", {"class": "profileCards__card "}) + \
                        carter_soup.findAll("div", {"class": "profileCards__card upgrade"}) + \
                        carter_soup.findAll("div", {"class": "profileCards__card"})

    carter_card_objs = [Card(card_html) for card_html in carter_cards_html]
    if verbose:
        print("Found {} cards".format(len(carter_card_objs)))
        
    try:
        assert len(carter_card_objs) == len(set(carter_card_objs))
    except AssertionError:
        print("Duplicate Cards Detected and removed")
        carter_card_objs = list(set(carter_card_objs))
    
    if verbose:
        print("After duplicate detection found {} cards".format(len(carter_card_objs)))

    carter_cards = [card_obj.to_row() for card_obj in carter_card_objs]
    df = pd.DataFrame(carter_cards); df.index = df.Name; del df["Name"]
    return df

In [6]:
carter_df = to_df(CARTER_KEY, verbose=True)
carter_df = carter_df.rename(columns={'Count': 'Carter Count', 'Level': 'Carter Level'})

chris_df = to_df(CHRIS_KEY, verbose=True)
chris_df = chris_df.rename(columns={'Count': 'Chris Count', 'Level': 'Chris Level'})

dad_df = to_df(DAD_KEY, verbose=True)
dad_df = dad_df.rename(columns={'Count': 'Dad Count', 'Level': 'Dad Level'})

Found 198 cards
After duplicate detection found 198 cards
Found 197 cards
After duplicate detection found 197 cards
Found 198 cards
After duplicate detection found 198 cards


### Writing out Scraped Data to an Excel File

In [12]:
def save_xls(list_dfs, df_names, xls_path):
    with ExcelWriter(xls_path) as writer:
        for name, df in zip(df_names, list_dfs):
            df.to_excel(writer, name)
        writer.save()

time_now = str(datetime.datetime.now())[:10]
save_xls([carter_df, chris_df, dad_df], ["carter_df", "chris_df", "dad_df"], "./clash_royale{}.xlsx".format(time_now))

### Trading Algorithm

- Preference Variables
    - Current level (maybe surplus)
    - Overall Popularity in the legendary arena

In [ ]:
def df_reader(player):
    return pd.read_excel('./clash_royale.xlsx', sheet_name='{}_df'.format(player))

In [ ]:
def get_joint(player_1, player_2):
    player_1_df, player_2_df = df_reader(player_1), df_reader(player_2)
    joint_df = pd.merge(player_1_df, player_2_df, left_on='Name', right_on='Name')
    del joint_df['Rarity_x']
    joint_df = joint_df.rename(columns={'Rarity_y' : 'Rarity'})
    joint_df = joint_df.drop_duplicates()
    joint_df['Level Diff'] = joint_df['{} Level'.format(player_1.title())].astype(int) -\
        joint_df['{} Level'.format(player_2.title())].astype(int)
    return joint_df

In [ ]:
def get_trades(joint_df, rarity, diff):
    joint_df = joint_df[(joint_df['Rarity'] == rarity) & (joint_df['Name_x'] == joint_df['Name_y'])]
    _1 = joint_df.loc[(joint_df["Level Diff"] == -1*diff)]
    _2 = joint_df.loc[(joint_df["Level Diff"] == diff)]
    return _1, _2

def get_rarity(joint_df, rarity):
    return joint_df[(joint_df['Rarity'] == rarity)]

In [ ]:
carter_and_dad = get_joint('carter', 'dad')
chris_and_dad = get_joint('chris', 'dad')
carter_and_chris = get_joint('carter', 'chris')

In [ ]:
get_rarity(carter_and_dad, "Epic").sort_values('Level Diff')